# DWDM LAB 3 Apriori Algorithm

In [1]:
import pandas as pd
import itertools

In [2]:
# Given
support_threshold = 2
DB_UNIQUE_ITEMS = []

In [3]:
db = [
    ['Bread', 'Milk'],
    ['Bread', 'Diapers', 'Beer', 'Eggs'],
    ['Milk', 'Diapers', 'Beer', 'Cola'],
    ['Bread', 'Milk', 'Diapers', 'Beer'],
    ['Bread', 'Milk', 'Diapers', 'Cola']
]

In [4]:
def support_count(items):
    count = 0
    for transaction in db:
        is_there = True
        for item in items:
            if item not in transaction:
                is_there = False
                break
        if is_there:
            count += 1
    return count

In [5]:
def get_candidate_sets(db, _n = None):    
    candidate_sets = []

    MAX_CANDIDATE_SET_LENGTH = _n
    if MAX_CANDIDATE_SET_LENGTH is None:
        # calculating item sets needed
        lns = [len(i) for i in db]
        MAX_CANDIDATE_SET_LENGTH = min(lns)

    temp_set = []
    # generate starter
    for transaction in db:
        for item in transaction:
            if item not in [x for i in temp_set for x in i[0]]:
                temp_set.append([[item], 0])
                DB_UNIQUE_ITEMS.append(item)

    #starting calculating candidate sets
    for __n in range(MAX_CANDIDATE_SET_LENGTH):
        flush_set = []

        # getting support count for current item_set
        for i, item in enumerate(temp_set):
            temp_set[i][1] = support_count(item[0])
        
        # removing items with support count less than support_threshold
        for item in temp_set:
            if item[1] >= support_threshold:
                flush_set.append(item)

        # appending to candidate_sets
        candidate_sets.append(flush_set)
        temp_set = [] # flushing temp_set for next iteration
        
        individual_items_final = list(set([x for i in flush_set for x in i[0] ]))

        # generating new temp_set
        for i in itertools.combinations(individual_items_final, __n + 2):
            keep=True
            for x in itertools.combinations(i, __n+1 ):
                if x not in candidate_sets[-1]:
                    if __n > 1:
                        keep=False
                    break
            if keep:
                temp_set.append([list(i), 0])

        

    return candidate_sets

candidate_sets = get_candidate_sets(db,3)
# candidate_sets

# Rules generations

In [6]:
def get_support(item_set, new_item):
    return support_count(item_set + new_item) / (len(db) if len(db) > 0 else 1)

In [7]:
def get_confidence(item_set, new_item):
    try:
        return support_count(item_set + new_item) / support_count(item_set)
    except ZeroDivisionError:
        return 0

In [8]:
def get_lift(item_set, new_item):
    try:
        return support_count(item_set + new_item) / (support_count(item_set) * support_count(new_item))
    except ZeroDivisionError:
        return 0

In [9]:
def get_rules(candidate_set, min_conf = 0.5):
    rules = []
    for items in candidate_set:
        subsets = [
            list(j) for i in 
            [list(itertools.combinations(items[0], x)) for x in range(1, len(items[0]) )] 
            for j in i
            ]
        items_set = set(items[0])
        for subset in subsets:
            lhs = subset
            rhs = list(items_set - set(lhs))
            lhs.sort()
            rhs.sort()
            _support, _confidence, _lift = get_support(lhs, rhs), get_confidence(lhs, rhs), get_lift(lhs, rhs)
            if _confidence >= min_conf :
                rules.append([",".join(lhs), ",".join(rhs), _support, _confidence, _lift])
    return rules


In [10]:
association_rules = get_rules(candidate_sets[-1])
rules = pd.DataFrame(association_rules, columns = ['LHS', 'RHS', 'Support', 'Confidence', 'Lift'])
rules

,LHS,RHS,Support,Confidence,Lift
0,Beer,"Diapers,Milk",0.4,0.666667,0.222222
1,Milk,"Beer,Diapers",0.4,0.500000,0.166667
2,Diapers,"Beer,Milk",0.4,0.500000,0.250000
3,"Beer,Milk",Diapers,0.4,1.000000,0.250000
4,"Beer,Diapers",Milk,0.4,0.666667,0.166667
5,"Diapers,Milk",Beer,0.4,0.666667,0.222222
6,Beer,"Bread,Diapers",0.4,0.666667,0.222222
7,Diapers,"Beer,Bread",0.4,0.500000,0.250000
8,Bread,"Beer,Diapers",0.4,0.500000,0.166667
9,"Beer,Diapers",Bread,0.4,0.666667,0.166667
